In [ ]:
import json

import pandas as pd
import smart_open

from gretel_client.projects import get_project, search_projects, models

## Projects Api

Note: This notebook assumes the host machine has been configured with Gretel Api credentials.

In [ ]:
for p in search_projects(limit=1):
    print(json.dumps(p.info(), indent=4, sort_keys=True))

In [ ]:
project = get_project(create=True)

## Models Api

In [ ]:
model = project.create_model(model_config="synthetics/default")

# this will override the datasource set from the template or blueprint
model.data_source = "https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/USAdultIncome5k.csv" 

In [ ]:
pd.read_csv(model.data_source)

In [ ]:
model.submit(runner_mode=models.RunnerMode.CLOUD, upload_data_source=True)

In [ ]:
for update in model.poll_logs_status():
    if update.transitioned:
        print(f"Status is {update.status}")
    for log in update.logs:
        msg = f"{log['ts']}  {log['msg']}"
        if log["ctx"]:
            msg += f"\n{json.dumps(log['ctx'], indent=4)}"
        print(msg)
    if update.error:
        print(f"\t{update.error}")

## Artifacts Api

In [ ]:
# list out all available artifacts and their download link
for art, link in model.get_artifacts():
    print(art, link)

In [ ]:
# read the data preview into a dataframe
df = pd.read_csv(model.get_artifact_link("data_preview"), compression='gzip')

In [ ]:
df.head()

In [ ]:
# download and parse the model report into a python object
with smart_open.open(model.get_artifact_link("report_json"), "rb") as rfd:
    report_json = json.loads(rfd.read())

report_json.keys()